**Create a complete list of all GALEX CAUSE Kepler survey visits.**

This script reads all mcat files relative to the specified root data directory and
extracts relevant information from the FITS header. A combined list for all mcat files
is created. All required information is contained that VASCA field and visits tables can
be created.

In [1]:
from tqdm.notebook import tqdm

In [2]:
import hashlib
import os
import pathlib

import numpy as np
import pandas as pd
import requests
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from datetime import datetime


import vasca.utils as vutils
import vasca.resource_manager as vascarm

# GALEX standard imaging mode visits table

In [3]:
rm = vascarm.ResourceManager()
tt_gal_vis = Table.read(rm.get_path("gal_visits_list","sas_cloud"))

In [4]:
display(tt_gal_vis[:10])
print(tt_gal_vis.info())

RATileCenter,DECTileCenter,survey,nexptime,fexptime,imgRunID,ParentImgRunID,joinID,tileNum,specTileNum,source,nPhotoObjects,nPhotoVisits,PhotoObsDate,spectra,nSpectra,nSpectraVisits,SpecObsDate,visitNum,subvis,minPhotoObsDate,maxPhotoObsDate,minSpecObsDate,maxSpecObsDate,PhotoObsDate_MJD,gall,galb
float64,float64,bytes3,float64,float64,int64,int64,int64,int64,int64,bytes6,int64,int64,bytes22,bytes5,bytes4,bytes4,bytes4,int64,int64,bytes22,bytes22,bytes4,bytes4,float64,float64,float64
210.542232477159,-32.6804067553693,NGS,544.0,544.0,2391589523080347648,2391589660720627712,7804,2437,-999,visitI,7781,1,6/7/2003 5:02:29 AM,False,null,null,null,1,-999,6/7/2003 5:02:29 AM,6/7/2003 5:11:33 AM,null,null,52797.21005787037,319.74270865062874,27.871918173555255
201.528614022379,-42.9957260941383,NGS,937.0,937.0,2391624707452436480,2391624845092716544,74460,2438,-999,visitI,8258,1,6/7/2003 6:41:02 AM,False,null,null,null,1,-999,6/7/2003 6:41:02 AM,6/7/2003 6:56:39 AM,null,null,52797.278495370374,309.64494588030084,19.423571577159674
203.769181415483,-30.085639855328,NGS,1349.05,1349.05,2391659891824525312,2391660029464805376,55314,2439,-999,visitI,10310,1,6/7/2003 8:19:41 AM,False,null,null,null,1,-999,6/7/2003 8:19:41 AM,6/7/2003 8:42:10 AM,null,null,52797.34700231482,314.0467834685246,31.84251982355922
231.921642929177,3.34027063665351,MIS,1696.0,1696.0,2391730260568702976,2391730398208983040,60377,2441,-999,visitI,11931,1,6/7/2003 11:37:03 AM,False,null,null,null,1,-999,6/7/2003 11:37:03 AM,6/7/2003 12:05:19 PM,null,null,52797.4840625,7.318115437067729,45.76107027896758
231.294710460205,2.47871950720032,MIS,1698.45,1698.45,2391765444940791808,2391765582581071872,80501,2442,-999,visitI,12210,1,6/7/2003 1:15:37 PM,False,null,null,null,1,-999,6/7/2003 1:15:37 PM,6/7/2003 1:43:55 PM,null,null,52797.552511574075,5.791997165545202,45.74212396120733
230.830932461494,3.23716734012699,MIS,1700.05,1700.05,2391800629312880640,2391800766953160704,19263,2443,-999,visitI,12072,1,6/7/2003 2:54:16 PM,False,null,null,null,1,-999,6/7/2003 2:54:16 PM,6/7/2003 3:22:36 PM,null,null,52797.62101851852,6.256101183947215,46.57064854646065
234.188554368289,16.8671059033759,NGS,1638.65,1638.65,2391835813684969472,2391835951325249536,43106,2444,-999,visitI,10393,1,6/7/2003 4:32:55 PM,False,null,null,null,1,-999,6/7/2003 4:32:55 PM,6/7/2003 5:00:13 PM,null,null,52797.68952546296,26.660767071324788,50.52230153629896
248.444951658959,28.9948431329317,NGS,1214.2,1214.15,2391870998057058304,2391871135697338368,66730,2445,-999,visitI,10950,1,6/7/2003 6:19:27 PM,False,null,null,null,1,-999,6/7/2003 6:19:27 PM,6/7/2003 6:39:41 PM,null,null,52797.763506944444,48.86491252912745,41.37504928770764
232.973573105411,3.52025544208571,MIS,820.05,820.05,2391906182429147136,2391906320069427200,41926,2446,-999,visitI,9171,1,6/7/2003 8:04:52 PM,False,null,null,null,1,-999,6/7/2003 8:04:52 PM,6/7/2003 8:18:32 PM,null,null,52797.83671296296,8.406902702835215,45.019198221613436


<Table length=100865>
      name        dtype 
---------------- -------
    RATileCenter float64
   DECTileCenter float64
          survey  bytes3
        nexptime float64
        fexptime float64
        imgRunID   int64
  ParentImgRunID   int64
          joinID   int64
         tileNum   int64
     specTileNum   int64
          source  bytes6
   nPhotoObjects   int64
    nPhotoVisits   int64
    PhotoObsDate bytes22
         spectra  bytes5
        nSpectra  bytes4
  nSpectraVisits  bytes4
     SpecObsDate  bytes4
        visitNum   int64
          subvis   int64
 minPhotoObsDate bytes22
 maxPhotoObsDate bytes22
  minSpecObsDate  bytes4
  maxSpecObsDate  bytes4
PhotoObsDate_MJD float64
            gall float64
            galb float64
None


# GCK example mcat

In [5]:
# VASCA columns - mcat header keys:
# "vis_id"
# "time_bin_start" - NEXPSTAR
# "time_bin_size" - NEXPTIME
# "sel"
# "obs_filter_id"
# "ra" - RA_CENT
# "dec" - DEC_CENT
# "gall" - GLONO
# "galb" - GLATO


In [6]:
def get_hdr_info(hdr):
    """
    Compile data from an mcat file header for general information about
    a drift scan observation.
    """
    # Extract header info
    hdr_info = {
        k: hdr[k]
        for k in [
            "TILENUM",
            "TILENAME",
            "OBJECT",
            "VISIT",
            "SUBVIS",
            "OBSDATIM",
            "NEXPSTAR",
            "NEXPTIME",
            "RA_CENT",
            "DEC_CENT",
            "GLONO",
            "GLATO",
        ]
    }

    # Create names and IDs for fields and visits
    field_name = f'{hdr_info["TILENUM"]}-{hdr_info["TILENAME"]}_sv{hdr_info["SUBVIS"]:02}'
    field_id = vutils.name2id(field_name, bits=64)
    vis_name = f'{field_name}_{hdr_info["VISIT"]:04}-img'
    vis_id = vutils.name2id(vis_name, bits=64)

    hdr_info.update(
        {
            "field_name": field_name,
            "field_id": field_id,
            "vis_name": vis_name,
            "vis_id": vis_id,
        }
    )

    # Time stamp
    time_bin_start = Time(hdr_info["NEXPSTAR"], format="unix").mjd
    hdr_info["time_bin_start"] = time_bin_start

    # Other info
    hdr_info.update(
        {
            "observatory": "GALEX_DS",  # GALEX drift scan
            "obs_filter": "NUV",
            "fov_diam": -999.99,  # FoV undefined for drift scan
            "sel": 0,
        }
    )

    return hdr_info

In [7]:
# Settings

# Input/output directories
root_data_dir = "/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields"
out_dir = "/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_visits_list"

# Dry-run, don't export final list
dry_run = False

# Loops over mcat files and saves info
info = list()
for path, subdirs, files in os.walk(root_data_dir):
    for name in files:
        if name.endswith("-xd-mcat.fits"):
            # Load mcat file and get relevant info
            mcat_path = os.path.join(path, name)
            with fits.open(mcat_path) as hdul:
                hdr_info = get_hdr_info(hdul[0].header)

            # Cross-checks
            # File name matches 'OBJECT' key
            if (
                mcat_path.split(os.sep)[-1].rstrip("-xd-mcat.fits")
                != hdr_info["OBJECT"]
            ):
                print(
                    "Warning: OBJECT key inconsistent "
                    f'(OBJECT: {hdr_info["OBJECT"]}, '
                    f'mcat: {mcat_path.split(os.sep)[-1].rstrip("-xd-mcat.fits")})'
                )
            # Visit directory name matches 'vis_name' key
            if mcat_path.split(os.sep)[-2] != hdr_info["vis_name"]:
                print(
                    "Warning: vis_name key inconsistent: "
                    f'(vis_name: {hdr_info["vis_name"]}, '
                    f"mcat directory: {mcat_path.split(os.sep)[-2]})"
                )

            info.append(hdr_info)

# Combines to astropy table via DataFrame
# because of problematic dtype handling of IDs
# tt_info = Table(info)  # this fails second cross-check
df_info = pd.DataFrame(info)
tt_info = Table.from_pandas(df_info)

# Cross-checks
# All visit IDs are unique
vis_ids = np.unique(tt_info["vis_id"])
if not len(vis_ids) == len(tt_info):
    raise ValueError("Non-unique visit IDs")
# All visit IDs have been consistently created from visit name
if not all(
    [
        int(vis_id) == vutils.name2id(vis_name, bits=64)
        for vis_id, vis_name in zip(tt_info["vis_id"], tt_info["vis_name"])
    ]
):
    raise ValueError("Inconsistent mapping vis_name to vis_id.")

if not dry_run:
    # Export
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)
    # FITS
    tt_info.write(f"{out_dir}/GALEX_DS_GCK_visits_list.fits", overwrite=True)
    # CSV
    df_info.to_csv(f"{out_dir}/GALEX_DS_GCK_visits_list.csv")
    # HTML
    df_info.to_html(f"{out_dir}/GALEX_DS_GCK_visits_list.html")

In [8]:
tt_info[:10]

TILENUM,TILENAME,OBJECT,VISIT,SUBVIS,OBSDATIM,NEXPSTAR,NEXPTIME,RA_CENT,DEC_CENT,GLONO,GLATO,field_name,field_id,vis_name,vis_id,time_bin_start,observatory,obs_filter,fov_diam,sel
int64,str15,str25,int64,int64,str14,float64,float64,float64,float64,float64,float64,str26,uint64,str35,uint64,float64,str8,str3,float64,int64
29208,KEPLER_SCAN_009,KEPLER_SCAN_009_0019_sv08,19,8,120907T090122Z,1347008481.995,228.35,291.69037852002,44.8728877632244,76.9861861593149,12.9961413692396,29208-KEPLER_SCAN_009_sv08,11563663594598293566,29208-KEPLER_SCAN_009_sv08_0019-img,4459283194592964374,56177.375949016205,GALEX_DS,NUV,-999.99,0
29208,KEPLER_SCAN_009,KEPLER_SCAN_009_0025_sv08,25,8,120919T015504Z,1348019703.995,121.0,291.69037852002,44.8728877632244,76.9861861593149,12.9961413692396,29208-KEPLER_SCAN_009_sv08,11563663594598293566,29208-KEPLER_SCAN_009_sv08_0025-img,15284256427265905955,56189.07990734954,GALEX_DS,NUV,-999.99,0
29208,KEPLER_SCAN_009,KEPLER_SCAN_009_0024_sv08,24,8,120918T192056Z,1347996055.995,118.0,291.69037852002,44.8728877632244,76.9861861593149,12.9961413692396,29208-KEPLER_SCAN_009_sv08,11563663594598293566,29208-KEPLER_SCAN_009_sv08_0024-img,8862978591103310281,56188.80620364583,GALEX_DS,NUV,-999.99,0
29208,KEPLER_SCAN_009,KEPLER_SCAN_009_0018_sv08,18,8,120906T163540Z,1346949339.995,229.05,291.69037852002,44.8728877632244,76.9861861593149,12.9961413692396,29208-KEPLER_SCAN_009_sv08,11563663594598293566,29208-KEPLER_SCAN_009_sv08_0018-img,12169717864554354965,56176.691435127315,GALEX_DS,NUV,-999.99,0
29208,KEPLER_SCAN_009,KEPLER_SCAN_009_0026_sv08,26,8,120919T033337Z,1348025616.995,121.0,291.69037852002,44.8728877632244,76.9861861593149,12.9961413692396,29208-KEPLER_SCAN_009_sv08,11563663594598293566,29208-KEPLER_SCAN_009_sv08_0026-img,8571124233770291412,56189.148344849535,GALEX_DS,NUV,-999.99,0
29208,KEPLER_SCAN_009,KEPLER_SCAN_009_0023_sv08,23,8,120917T035601Z,1347854160.995,118.0,291.69037852002,44.8728877632244,76.9861861593149,12.9961413692396,29208-KEPLER_SCAN_009_sv08,11563663594598293566,29208-KEPLER_SCAN_009_sv08_0023-img,13212896263789748751,56187.16390040509,GALEX_DS,NUV,-999.99,0
29208,KEPLER_SCAN_009,KEPLER_SCAN_009_0022_sv08,22,8,120910T091834Z,1347268713.995,220.05,291.69037852002,44.8728877632244,76.9861861593149,12.9961413692396,29208-KEPLER_SCAN_009_sv08,11563663594598293566,29208-KEPLER_SCAN_009_sv08_0022-img,16143993856861089493,56180.38789346065,GALEX_DS,NUV,-999.99,0
29208,KEPLER_SCAN_009,KEPLER_SCAN_009_0020_sv08,20,8,120907T153538Z,1347032137.995,229.05,291.69037852002,44.8728877632244,76.9861861593149,12.9961413692396,29208-KEPLER_SCAN_009_sv08,11563663594598293566,29208-KEPLER_SCAN_009_sv08_0020-img,15804500812357242501,56177.6497453125,GALEX_DS,NUV,-999.99,0
29208,KEPLER_SCAN_009,KEPLER_SCAN_009_0021_sv08,21,8,120908T080119Z,1347091278.995,222.05,291.69037852002,44.8728877632244,76.9861861593149,12.9961413692396,29208-KEPLER_SCAN_009_sv08,11563663594598293566,29208-KEPLER_SCAN_009_sv08_0021-img,16384408983742434967,56178.33424762731,GALEX_DS,NUV,-999.99,0
